In [1]:
import numpy as np
import mcubes
import nibabel
from skimage.transform import resize
import copy
import os
from os.path import join
import SimpleITK as sitk

In [18]:
SAVE_OBJ_FOLD = r'G:\front_web\cudes\B-obj'
SAVE_NII_FOLD = r'G:\front_web\cudes\A-nii'
DATASET_FOLD = r'D:\juputer\CECT\ground_truth\clear'

In [2]:
def dcm2nii_sitk(path_read, path_save,file_name):
    reader = sitk.ImageSeriesReader()
    seriesIDs = reader.GetGDCMSeriesIDs(path_read)

    N = len(seriesIDs)
    lens = np.zeros([N])
    for i in range(N):
        dicom_names = reader.GetGDCMSeriesFileNames(path_read, seriesIDs[i])
        lens[i] = len(dicom_names)
    N_MAX = np.argmax(lens)
    dicom_names = reader.GetGDCMSeriesFileNames(path_read, seriesIDs[N_MAX])

    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    if not os.path.exists(path_save):
        os.mkdir(path_save)
    sitk.WriteImage(image, join(path_save,file_name+'.nii.gz'))

def comebine_2_nii(source_path1,source_path2,file_name):
    # 读取nii文件
    nii_img = nibabel.load(source_path1)
    nii_data = nii_img.get_fdata()
    # new_data = nii_data.copy()
    nii_data2 = nibabel.load(source_path2).get_fdata()

    # 省略一些处理data的骚操作,比如：
    new_data = nii_data+nii_data2
    new_data = new_data.astype(np.uint8)

    # 把仿射矩阵和头文件都存下来
    affine = nii_img.affine.copy()
    hdr = nii_img.header.copy()

    # 形成新的nii文件
    new_nii = nibabel.Nifti1Image(new_data, affine, hdr)

    # 保存nii文件，后面的参数是保存的文件名
    nibabel.save(new_nii, file_name+'.nii.gz')


In [44]:
dcm2nii_sitk(r'D:\juputer\CECT\ground_truth\clear\zongjinzhang_pre_CT\pelvis_artery',r'G:\front_web\cudes\nii','zongjinzhang_pre_CT')

In [8]:
patient = 'zhangyuejian_pre_CT'
dcm2nii_sitk(join(DATASET_FOLD,patient,'pelvis_artery'),SAVE_NII_FOLD,patient)

In [15]:
def resize_segmentation(segmentation, new_shape, order=3, cval=0):
    '''
    Resizes a segmentation map. Supports all orders (see skimage documentation). Will transform segmentation map to one
    hot encoding which is resized and transformed back to a segmentation map.
    This prevents interpolation artifacts ([0, 0, 2] -> [0, 1, 2])
    :param segmentation:
    :param new_shape:
    :param order:
    :return:
    '''
    tpe = segmentation.dtype
    unique_labels = np.unique(segmentation)
    assert len(segmentation.shape) == len(new_shape), "new shape must have same dimensionality as segmentation"
    reshaped = np.zeros(new_shape, dtype=segmentation.dtype)
    for i, c in enumerate(unique_labels):
        mask = segmentation == c
        reshaped_multihot = resize(mask.astype(float), new_shape, order, mode="edge", clip=True, anti_aliasing=False)
        reshaped[reshaped_multihot >= 0.5] = c
        print(i)
    return reshaped

def read_file_and_resize(file_path):
    if not file_path.endswith('.nii.gz'):
        file_path = file_path+'.nii.gz'
    file = nibabel.load(file_path)
    img = file.get_fdata()
    spacing = np.array(file.header.get_zooms())
    position_array = np.where(img)
    start = np.array(position_array).min(1)
    end = np.array(position_array).max(1)
    print('file have been read.')
    cube = img[start[0]:end[0]+2,start[1]:end[1]+2,start[2]:end[2]+2]
    shape = np.array(cube.shape)
    new_shape = np.around(spacing/spacing.min()*shape).astype(int)
    print('resize the spacing into the same....there need some time')
    new_array = resize_segmentation(cube,new_shape)
    return new_array

def get_multi_class_reconstruction(file_path,save_path,zxy_position=[2,1,0],parts=None):
    new_array = read_file_and_resize(file_path)
    z,x,y = zxy_position
    array = np.transpose(new_array,[y,z,x])

    if not os.path.exists(save_path):
        os.mkdir(save_path)

    if parts==None:
        parts = ['back','pelvis','aorta','right_common_iliac_artery','left_common_iliac_artery','right_external_iliac_artery','right_internal_iliac_artery','left_external_iliac_artery','left_internal_iliac_artery']
    for index in range(1,len(parts)):
        test = copy.deepcopy(array)
        test[test!=index]=0
        test[test==index]=1
        not_zero = np.where(test)
        start = np.array(not_zero).min(1)
        end = np.array(not_zero).max(1)
        part = test[start[0]:end[0]+2,start[1]:end[1]+2,start[2]:end[2]+2]
        part = mcubes.smooth(part,'constrained')
        test[start[0]:end[0]+2,start[1]:end[1]+2,start[2]:end[2]+2] = part
        vertices, triangles = mcubes.marching_cubes(test, 0)
        mcubes.export_obj(vertices, triangles, os.path.join(save_path,parts[index]+'.obj'))
        print(parts[index]+' have done')


In [17]:
get_multi_class_reconstruction(join(SAVE_NII_FOLD,patient),join(SAVE_OBJ_FOLD,patient))

file have been read.
resize the spacing into the same....there need some time
0
1
2
3
4
5
6
7
8
pelvis have done
aorta have done
right_common_iliac_artery have done
left_common_iliac_artery have done
right_external_iliac_artery have done
right_internal_iliac_artery have done
left_external_iliac_artery have done
left_internal_iliac_artery have done


In [ ]:
# use node to change the obj to gltf
# node change2gltf.js

In [1]:
SAVE_GLTF_FOLD=r'G:\front_web\cudes\C-module'
for i in os.listdir('B-obj'):
    try:
        os.makedirs(join(SAVE_GLTF_FOLD,i.split('.')[0]))
    except:
        continue

In [2]:
os.listdir(SAVE_GLTF_FOLD)

['huangzeming_pre_CT',
 'lvlong_pre_CT',
 'maokuirong_pre_CT',
 'wuxingcai_pre_CT',
 'xuchaozhi_pre_CT',
 'xuwentou_pre_CT',
 'yaokai_pre_CT']